# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли уровень дохода,  семейное положение и количество детей клиента, а так же цели на которые берется заем на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


При обзоре можно обнаружить пропуски в двух стобцах days_employed и total_income      

### Шаг 2.1 Заполнение пропусков

In [2]:
print(df.isna().sum() / df.shape[0])

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64


In [3]:
median_total = df['total_income'].median()
print('Значение медианы равно', median_total)
df['total_income'] = df['total_income'].fillna(median_total)
print(df['total_income'].isna().sum())
display(df.head(10))

Значение медианы равно 145017.93753253992
0


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


1. В стоблце 'total_income' было обнаружено 2174 пропущенных значений типа NaN.
2. По скольку в столбце отражен ежемесячный доход, пропущенный значения повлияют на конечный итог анализа, поэтому было принято решение заполнить пропущенные значения на медианное значение, согласно столбцу 
3. Проверил на наличие пропусков, командой info()
4. Доля пропущенных значений в столбце days_employed = 10% total_income = 10%

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [4]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце 'children' обнаружили аномалии в виде отрицальных значений, а также в количестве детей равном 20

In [5]:
df.loc[df['children'] == 20, 'children'] = 2
df['children'] = abs(df['children'])
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [6]:
df['days_employed'] = abs(df['days_employed'])
days_emloyed_med = df['days_employed'].median()
df['days_employed'] = df['days_employed'].fillna(value = days_emloyed_med)
df['days_employed'].isna().sum()
print(df['days_employed'])

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64


Аномалии были обнаружены так же в столбце days_employed, по мимо пропущенных значнений, заполненных медианой, так же были обнаружены отрицальные значения, что вполне могло произойти из-за измения формата таблицы, к примеру.
И подозрительно больше значение вроде 343937 дней, в данном этапе это произошло уже скорее из-за человеческого фактора. Но по скольку столбец в анализе использован не будет, это можно не учитывать. Опять же в целях экономии времени

### Шаг 2.3. Изменение типов данных.

In [7]:
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Для упрощения анализа, избавились от дробных чисел в столбце с заработком, изменив тип с float на int

### Шаг 2.4. Удаление дубликатов.

In [8]:
df['education'] = df['education'].str.lower()
print('Количество дупликатов до удаления:', df.duplicated().sum())

Количество дупликатов до удаления: 71


In [9]:
df = df.drop_duplicates()
print(df['education'].unique())
print('Количество дупликатов после удаления:', df.duplicated().sum())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
Количество дупликатов после удаления: 0


с помощью команды drop_duplicates(), удаляем дупликаты в таблицы, и приводим строки к единому регистру,
проверили работу команд с помощью метода unique() и duplicated().sum()

1. Выбрал метод drop_duplicates чтобы удалить все дубликаты из дата фрейма, атрибут inplace = True, для того что переименовать данные
2. Дубликаты могут появиться по двум причинам, - Человеческий фактор и Связаннный непосредственно с системой, например когда переводим из одного формата в другой.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [10]:
education = display(df[['education', 'education_id']].drop_duplicates().reset_index(drop=True))
family = display(df[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop=True))
df = df.drop(columns = ['education', 'family_status'])
display(df)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем
21521,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость
21523,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля


Упростим анализ, удалив два столбца, поместив их в словари.

### Шаг 2.6. Категоризация дохода.

In [11]:
def income_category(rub):
    if rub <= 30000:
        return 'E'
    if rub <= 50000:
        return 'D'
    if rub <= 200000:
        return 'C'
    if rub <= 1000000:
        return 'B'
    return 'A'
df['total_income_category'] = df['total_income'].apply(income_category)
df_pivot = df.pivot_table(index=['total_income_category', 'debt'], columns='children', values='total_income', aggfunc='count')
display(df_pivot) 


children                         0       1       2      3     4    5
total_income_category debt                                          
A                     0       11.0     8.0     3.0    1.0   NaN  NaN
                      1        1.0     1.0     NaN    NaN   NaN  NaN
B                     0     3025.0  1087.0   479.0   84.0   6.0  4.0
                      1      220.0    88.0    43.0    4.0   1.0  NaN
C                     0     9746.0  3248.0  1413.0  214.0  31.0  4.0
                      1      828.0   352.0   155.0   22.0   3.0  NaN
D                     0      231.0    63.0    30.0    4.0   NaN  1.0
                      1       12.0     4.0     4.0    1.0   NaN  NaN
E                     0       15.0     4.0     1.0    NaN   NaN  NaN
                      1        2.0     NaN     NaN    NaN   NaN  NaN

### Шаг 2.7. Категоризация целей кредита.

In [12]:

def purpose_category(cat):
    if 'автомоб' in cat:
        return 'операции с автомобилем',
    if 'недвиж' in cat:
        return 'операции с недвижимостью',
    if 'жиль' in cat:
        return 'операции с недвижимостью',
    if 'свадьб' in cat:
        return 'проведение свадьбы',
    if 'образов' in cat:
        return 'получение образования'
df['purpose_category'] = df['purpose'].apply(purpose_category)
df_pivot_purpose = df.pivot_table(index=['purpose_category', 'debt'], columns='total_income_category', values='total_income', aggfunc='count')
display(df_pivot_purpose) 


total_income_category                A       B       C      D    E
purpose_category            debt                                  
(операции с автомобилем,)   0      2.0   945.0  2892.0   62.0  2.0
                            1      NaN    86.0   307.0   10.0  NaN
(операции с недвижимостью,) 0     16.0  2396.0  7446.0  162.0  9.0
                            1      1.0   172.0   601.0    6.0  2.0
получение образования       0      3.0   827.0  2734.0   74.0  5.0
                            1      1.0    70.0   297.0    2.0  NaN
(проведение свадьбы,)       0      2.0   517.0  1584.0   31.0  4.0
                            1      NaN    28.0   155.0    3.0  NaN

### Шаг 2.2. Построим дополнительно таблицу с зависимостью от брака

In [13]:
df_pivot_married = df.pivot_table(index=['total_income_category', 'debt'], columns='family_status_id', values='total_income', aggfunc='count')
display(df_pivot_married) 

family_status_id                 0       1      2      3       4
total_income_category debt                                      
A                     0       13.0     5.0    NaN    3.0     2.0
                      1        1.0     1.0    NaN    NaN     NaN
B                     0     2757.0   912.0  147.0  264.0   605.0
                      1      203.0    64.0    9.0   18.0    62.0
C                     0     8415.0  2783.0  726.0  828.0  1904.0
                      1      713.0   318.0   54.0   65.0   210.0
D                     0      211.0    57.0   23.0   15.0    23.0
                      1       12.0     5.0    NaN    2.0     2.0
E                     0       12.0     6.0    NaN    NaN     2.0
                      1        2.0     NaN    NaN    NaN     NaN

### Ответы на вопросы.

##### Вопрос 1: # Есть ли зависимость между количеством детей и возвратом кредита в срок?

Согласно таблице 'df_pivot' можно сделать вывод, количество детей и задолжность по кредиту имеет зависимость. На людей у которых нет ни одного ребенка, самое большое количество задолжностей по всем категориям. Число задолжностей начинает снижается пропорцианально возрастанию количества детей в семье.

##### Вопрос 2: # Есть ли зависимость между семейным положением и возвратом кредита в срок?

Женатые люди имеют самое большое число задолжностей, так же видно что задолжности снижаются согласно id таблицы и располагаются в таком порядке
1. Женатые
2. Гражданский брак
3. Вдовы
4. Разведенные
5. Не женатые

##### Вопрос 3: # Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Самое большое количество задолжностей наблюдается в категории "С" тоесть доход от 50000 до 200000, но это вполне может быть обусловлено так же тем, что в категории "С" больше чем в остальный берут кредит. А вот уже от категории "С" в сторону "Е" количетсво резко уменьшается вплоть до полного неимения задолжности, а в сторону категории "А" тоже уменьшается но более плавно. Поэтому тут можно сказать, что чем меньше заработок у людей, тем меньше они берут кредиты, а вот уже от заработка категорий "С" и "В" можно исходить, что такие доходы чаще всего являются потенциальными клиентами на кредит.

##### Вопрос 4: # Как разные цели кредита влияют на его возврат в срок?

Самое высокое количество задолжностей приходиться на кредиты на недвижимость, что скорее всего вызвано длительностью таких кредитов. Самое минимальное на проведение свадеб и уже потом только образование и следом покупка автомобиля.

##### Вывод 1:

Анализ показал, что самыми частыми заемщиками являются люди с заработком категории "C", так же что у женатых людей наиболее часто встречаются задолжности в отличии от не женатых, наличие детей не является фактором, по которому можно судить о том, что у заемщика будут задолжности, а так же цели на кредит, исходят чаще всего из своей длительности, к примеру кредиты на недвижимость в силу своей большей стоимости, растягиваются и на большее время, в отличии например от кредитов на свадьбы. 

## Общий вывод:

Было проведен общий обзор данных, в котором выявили пропущенные значения в двух столбцах: days_employed и total_income. Затем заменили пропущенные значения на медианные значения по столбам. Затем проверили столбцы на аномалии, которые были обнаружены в children, days_employed и education, отрицательные значения, буквы разных регистров, а так же дупликаты и ошибочно введенные значения, были исправлены. После этого декомпозировали столбцы education и family_status, в отдельные словари, а в основной таблицы оставили только id этих столбцов, чтобы можно было упростить и продолжить анализ. И в завершении анализы категоризировали данные в зависимостях от доходов, наличия детей и брака, что позволило найти закономерности и привести следующий вывод.
На основании всех полученных данных можно сделать общий вывод, что длительность кредита и его величина, вполне может повлиять на возникновении задолжностей независимо от заработка заемщика. А так же, что размер кредита напрямую зависит от заработков заемщиков.